In [24]:
import pandas as pd
import numpy as np
import spacy
from nltk import tokenize
import googletrans
from googletrans import Translator
import time
import math
import jieba
import random
import urllib
import requests
import csv
import gzip
from collections import defaultdict

In [85]:
df = pd.read_csv('seame_data_annotated.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,row_number,original_row_number,file,type,start_time,end_time,transcript_2014,filtered_transcript_2014,transcript_2015,filtered_transcript_2015,...,Maximum pitch,Minimum intensity,Mean intensity,Maximum intensity,Jitter,Shimmer,HNR,Speaking rate,Std Dev Pitch,Std Dev Intensity
0,2,NaN,01NC01FBX_0101,Conversation,0,6010,NaN,NaN,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,9360.0,01NC02FBY_0101,Conversation,0,1470,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,NaN,01NC02FBY_0101,Conversation,2201,2605,NaN,NaN,我们,['我们'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,NaN,01NC02FBY_0101,Conversation,3037,3804,NaN,NaN,进 [啊] 没有,"['进', '没有']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,01NC02FBY_0101,Conversation,6675,7583,NaN,NaN,(ppl) 等一下 (ppb),"['等', '一下']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
len(df)

110145

In [87]:
df.columns

Index(['row_number', 'original_row_number', 'file', 'type', 'start_time',
       'end_time', 'transcript_2014', 'filtered_transcript_2014',
       'transcript_2015', 'filtered_transcript_2015', 'length_of_turn',
       'start_of_code_switches', 'length_of_code_switched_segments',
       'direction_of_code_switches', 'number_of_code_switches',
       'intersentential_code_switch?', 'dialog_act', 'type_of_code_switching',
       'language', 'conversation', 'speaker', 'Minimum pitch', 'Mean pitch',
       'Maximum pitch', 'Minimum intensity', 'Mean intensity',
       'Maximum intensity', 'Jitter', 'Shimmer', 'HNR', 'Speaking rate',
       'Std Dev Pitch', 'Std Dev Intensity'],
      dtype='object')

In [88]:
df = df[['file', 'type','transcript_2015', 'filtered_transcript_2015', 'length_of_turn',
       'start_of_code_switches', 'length_of_code_switched_segments',
       'direction_of_code_switches', 'number_of_code_switches',
       'intersentential_code_switch?','type_of_code_switching','language', 'conversation', 'speaker']]
df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2
2,01NC02FBY_0101,Conversation,我们,['我们'],1,-1,-1,-1,0,-1,-1,ZH,1,2
3,01NC02FBY_0101,Conversation,进 [啊] 没有,"['进', '没有']",2,-1,-1,-1,0,-1,-1,ZH,1,2
4,01NC02FBY_0101,Conversation,(ppl) 等一下 (ppb),"['等', '一下']",2,-1,-1,-1,0,-1,-1,ZH,1,2


In [5]:
csw_df = df[df['language']=='CS']
csw_df = csw_df[csw_df['length_of_turn']!=0]
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1
14,01NC02FBY_0101,Conversation,[哦] cause 那个 #cui#,"['cause', '那个', '#cui#']",3,"[1, 2]","[1, 1]","['eng > man', 'man > other']",2,N,I,CS,1,2


In [6]:
len(csw_df)

56906

In [7]:
marking = []
for lst in csw_df['direction_of_code_switches']:
    #print(type(lst))
    as_list = lst.strip('][').split(', ')
    #print(as_list)
    if as_list[0] == "'man > eng'":
        marking.append('Y')
    else:
        marking.append('N')
csw_df['marking'] = marking
csw_df = csw_df[csw_df['marking']=='Y']
len(csw_df)

34117

In [8]:
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y


In [9]:
first_cs_word = []
for index, row in csw_df.iterrows():
    lst = row['start_of_code_switches']
    as_list = lst.strip('][').split(', ')
    idx = int(as_list[0])
    word = row['filtered_transcript_2015'].strip('][').split(', ')[idx]
    if word.startswith("'") and word.endswith("'"):
        word = word[1:-1]
    first_cs_word.append(word)
csw_df['first_cs_word'] = first_cs_word

In [10]:
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top


In [11]:
nlp = spacy.load("en_core_web_sm")

In [12]:
tags = []
for word in list(csw_df['first_cs_word']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break

In [13]:
coded_POS = []
for tag in tags:
    if tag == 'NN' or tag == 'NNP' or tag == 'NNS' or tag == 'NNPS':
        coded_POS.append(0)
    elif 'V' in tag:
        coded_POS.append(1)
    else:
        coded_POS.append(2)

In [14]:
csw_df['first_cs_word_eng_pos'] = coded_POS
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2


In [15]:
word_length = []
for word in list(csw_df['first_cs_word']):
    word_length.append(len(word))
csw_df['word_length_eng'] = word_length
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3


In [15]:
#!pip install googletrans==4.0.0rc1
print(googletrans.LANGUAGES) #check language codes
print(googletrans.__version__) #make sure this is 4.0.0-rc.1
translator = Translator()
translator.raise_Exception = True

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [55]:
en_translated_lines = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)
    
    try:
        result = translator.translate(as_str, src='zh-cn', dest='en').text
        time.sleep(0.3)
        en_translated_lines.append(result.lower())
    except:
        en_translated_lines.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [56]:
en_translated_lines.count('TODO')

169

In [57]:
en_translated_lines.index('TODO')

1146

In [58]:
csw_df['english_translation'] = en_translated_lines
csw_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation
0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ..."
1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3,october you mean finals
5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah
7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that..."
18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3,because top four


In [59]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [3]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,-1,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ..."
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,N,I,CS,1,2,Y,you,2,3,october you mean finals
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,N,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,Y,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that..."
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,N,O,CS,1,1,Y,top,2,3,because top four


In [4]:
len(csw_df)

34117

In [11]:
first_cs_word_eng = list(csw_df['first_cs_word'])
first_cs_word_zh = []
for i in range(len(first_cs_word_eng)):
    word = first_cs_word_eng[i]
    if i%100==0:
        print(i)
    try:
        result = translator.translate(word, src='en', dest='zh-cn').text
        time.sleep(0.3)
        first_cs_word_zh.append(result)
    except:
        first_cs_word_zh.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [12]:
first_cs_word_zh.count('TODO')

6

In [13]:
csw_df['first_cs_word_zh'] = first_cs_word_zh
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,...,type_of_code_switching,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,...,I,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,...,I,CS,1,2,Y,you,2,3,october you mean finals,你
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,...,-1,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,...,I,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,...,O,CS,1,1,Y,top,2,3,because top four,顶部


In [22]:
first_cs_word_zh_len = []
for word in first_cs_word_zh:
    first_cs_word_zh_len.append(len(word))
print(max(first_cs_word_zh_len))

22


In [26]:
csw_df['word_length_zh'] = first_cs_word_zh_len
csw_df.head()

,Unnamed: 0,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,...,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh
0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",2,...,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1
1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],1,...,CS,1,2,Y,you,2,3,october you mean finals,你,1
2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],1,...,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2
3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",8,...,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1
4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],1,...,CS,1,1,Y,top,2,3,because top four,顶部,2


In [33]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [4]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,...,language,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh
0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",...,CS,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1
1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],...,CS,1,2,Y,you,2,3,october you mean finals,你,1
2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],...,CS,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2
3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",...,CS,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1
4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],...,CS,1,1,Y,top,2,3,because top four,顶部,2


In [149]:
nlp = spacy.load("zh_core_web_sm")

In [8]:
tags = []
for word in list(csw_df['first_cs_word_zh']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break

In [9]:
coded_POS = []
for pos in tags:
    if pos == 'NN':
        coded_POS.append(0)
    elif pos == 'NR':
        coded_POS.append(0)
    elif pos == 'NT':
        coded_POS.append(0)
    elif pos == 'VE':
        coded_POS.append(1)
    elif pos == 'VV':
        coded_POS.append(1)
    else:
        coded_POS.append(2)

In [10]:
csw_df['first_cs_word_zh_pos'] = coded_POS
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,...,conversation,speaker,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos
0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"[10, 11]","[1, 4]","['man > eng', 'eng > man']",...,1,1,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2
1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,[1],[3],['man > eng'],...,1,2,Y,you,2,3,october you mean finals,你,1,2
2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,[5],[1],['man > eng'],...,1,2,Y,yah,0,3,can't tell whether we are yah,雅虎,2,0
3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"[6, 7, 14, 15, 18, 21, 28, 29]","[1, 7, 1, 3, 3, 7, 1, 1]","['man > eng', 'eng > man', 'man > eng', 'eng >...",...,1,1,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1,2
4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,[1],[2],['man > eng'],...,1,1,Y,top,2,3,because top four,顶部,2,0


In [11]:
csw_df.to_csv('seame_with_eng_translations.csv')

In [35]:
csw_df = pd.read_csv('seame_with_eng_translations.csv')
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,marking,first_cs_word,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,Y,but,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,Y,you,2,3,october you mean finals,你,1,2,950006304,25465651
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,Y,yah,0,3,can't tell whether we are yah,雅虎,2,0,131496,11734
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,Y,but,2,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,Y,top,2,3,because top four,顶部,2,0,40791068,0


In [53]:
first_cs_word_eng_freq = []
first_cs_word_zh_freq = []

total_count_eng = 356033418959
total_count_zh = 26859461025

first_cs_word_eng_raw_freq = list(csw_df['first_cs_word_eng_raw_freq'])
first_cs_word_zh_raw_freq = list(csw_df['first_cs_word_zh_raw_freq'])

for freq in first_cs_word_eng_raw_freq:
    if freq=='TODO':
        first_cs_word_eng_freq.append(freq)
    else:
        if int(freq)==0:
            first_cs_word_eng_freq.append(-math.log2(1/total_count_eng))
        else:
            first_cs_word_eng_freq.append(-math.log2(int(freq)/total_count_eng))
        

for freq in first_cs_word_zh_raw_freq:
    if freq=='TODO':
        first_cs_word_zh_freq.append(freq)
    else:
        if int(freq)==0:
            first_cs_word_zh_freq.append(-math.log2(1/total_count_zh))
        else:
            first_cs_word_zh_freq.append(-math.log2(int(freq)/total_count_zh))
        
csw_df['first_cs_word_eng_freq'] = first_cs_word_eng_freq
csw_df['first_cs_word_zh_freq'] = first_cs_word_zh_freq

csw_df.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,first_cs_word_eng_pos,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,2,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.812799,9.104577
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,2,3,october you mean finals,你,1,2,950006304,25465651,8.54986,10.042662
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,0,3,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562,21.126304
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,2,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.812799,9.104577
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,2,3,because top four,顶部,2,0,40791068,0,13.091472,34.644711
5,5,5,5,5,22,01NC02FBY_0101,Conversation,对 [呀] 那个 area five 多少,"['对', '那个', 'area', 'five', '多少']",5,...,0,4,how much is area five,区域,2,0,64951337,5054553,12.420366,12.375559
6,6,6,6,6,24,01NC02FBY_0101,Conversation,[诶] 那个 then 谁 拿 那个 top one 那个 foucia*,"['那个', 'then', '谁', '拿', '那个', 'top', 'one', '...",9,...,2,4,that then who takes that top one that foucia*,然后,2,2,283578912,5126199,10.294047,12.355253
7,7,7,7,7,31,01NC01FBX_0101,Conversation,因为 我 一直 bug 他 讲 [诶] result 几时 会 出 r...,"['因为', '我', '一直', 'bug', '他', '讲', 'result', '...",15,...,0,3,"because i have been bug, he said that result w...",漏洞,2,0,1082078,226962,18.327849,16.852620
8,8,8,8,8,35,01NC01FBX_0101,Conversation,就是 score 就是 score [mah] then 为什么 要 then...,"['就是', 'score', '就是', 'score', 'then', '为什么', ...",21,...,1,5,it is score that is the score then why he said...,分数,2,0,9348150,74680,15.216972,18.456277
9,9,9,9,9,36,01NC02FBY_0101,Conversation,[哎哟] 哪里 可能 那个 你 不 就 跟 我 讲 tricia 拿 ...,"['哪里', '可能', '那个', '你', '不', '就', '跟', '我', '讲...",13,...,0,6,where can you tell me about tricia and take on...,Tricia,6,0,127876,0,21.408836,34.644711


In [54]:
zh_translated_lines = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)
    
    try:
        result = translator.translate(as_str, src='en', dest='zh-cn').text
        time.sleep(0.3)
        zh_translated_lines.append(result)
    except:
        zh_translated_lines.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [55]:
zh_translated_lines.count('TODO')

30

In [56]:
csw_df['chinese_translation'] = zh_translated_lines
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,...,word_length_eng,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation
0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,...,3,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去
1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",4,...,3,october you mean finals,你,1,2,950006304,25465651,8.54986,10.042662,十月你是说决赛
2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,...,3,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是
3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,...,3,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...
4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",3,...,3,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名


In [57]:
csw_df.to_csv('seame_with_eng_zh1_translations.csv')

In [3]:
csw_df = pd.read_csv('seame_with_eng_zh1_translations.csv')

In [28]:
def merge_spaces(list_of_strings):
    for i in range(len(list_of_strings)):
        try:
            if list_of_strings[i] == '':
                while i < len(list_of_strings)-1 and list_of_strings[i+1]=='':
                    list_of_strings.pop(i+1)
        except:
            pass
    return list_of_strings  

In [29]:
def merge_words(list_of_eng_strings):
    for i in range(len(list_of_eng_strings)):
        try:
            if list_of_eng_strings[i] != '':
                while i < len(list_of_eng_strings)-1 and list_of_eng_strings[i+1]!='':
                    list_of_eng_strings[i] = list_of_eng_strings[i] + ' ' + list_of_eng_strings[i+1]
                    list_of_eng_strings.pop(i+1)

        except:
            pass
    list_of_eng_strings = [word for word in list_of_eng_strings if word!='']
    return list_of_eng_strings

In [74]:
zh_trans_word_by_word = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
        
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)

    eng_only = []
    zh_only = []
    for word in as_list:
        if word.isascii():
            eng_only.append(word)
            zh_only.append('')
        else:
            zh_only.append(word)
            eng_only.append('')
    
    zh_only = merge_spaces(zh_only)
    eng_only = merge_spaces(eng_only)
    
    try:
        for word in merge_words(eng_only):
            result = translator.translate(word, src='en', dest='zh-cn').text
            zh_only[zh_only.index('')] = result
        time.sleep(0.3)
        final_result = ''.join(zh_only)
        zh_trans_word_by_word.append(' '.join(jieba.lcut(final_result)))
        
    except:
        zh_trans_word_by_word.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [75]:
zh_trans_word_by_word.count('TODO')

926

In [76]:
csw_df['chinese_wbw_translation'] = zh_trans_word_by_word
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,...,english_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,"uh, on october 11th, i tell you a thing to be ...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",...,can't tell whether we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",...,"let me tell you one thing, but we can say that...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [77]:
csw_df.to_csv('seame_with_eng_zh2_translations.csv')

In [78]:
en_trans_word_by_word = []
for i in range(len(list(csw_df['filtered_transcript_2015']))):
    if i%100==0:
        print(i)
        
    ini_list = list(csw_df['filtered_transcript_2015'])[i]
    as_list = ini_list.strip('][').split(', ')
    as_list = [word[1:-1] for word in as_list if word.startswith("'") and word.endswith("'")]
    as_str = ' '.join(as_list)

    eng_only = []
    zh_only = []
    for word in as_list:
        if word.isascii():
            eng_only.append(word)
            zh_only.append('')
        else:
            zh_only.append(word)
            eng_only.append('')
    
    zh_only = merge_spaces(zh_only)
    eng_only = merge_spaces(eng_only)
    
    try:
        for word in merge_words(zh_only):
            result = translator.translate(word, src='zh-cn', dest='en').text
            eng_only[eng_only.index('')] = result
        time.sleep(0.3)
        final_result = ' '.join(eng_only)
        en_trans_word_by_word.append(final_result.lower())
        
    except:
        en_trans_word_by_word.append('TODO')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [79]:
en_trans_word_by_word.count('TODO')

4724

In [80]:
csw_df['english_wbw_translation'] = en_trans_word_by_word
csw_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,file,type,transcript_2015,filtered_transcript_2015,...,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,english_wbw_translation
0,0,0,0,0,0,0,01NC01FBX_0101,Conversation,(呃) 十一号 十月 哦 我 跟 你 讲 一个 东西 but 不 可以 讲 出去,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",...,但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,"uh, on october 11th, i tell you a thing to tel..."
1,1,1,1,1,1,1,01NC02FBY_0101,Conversation,十月 you mean finals,"['十月', 'you', 'mean', 'finals']",...,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,october you mean finals
2,2,2,2,2,2,5,01NC02FBY_0101,Conversation,不可以 讲出去 (ppl) 我们 是不是 (ppl) yah,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",...,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,can't tell if we are yah
3,3,3,3,3,3,7,01NC01FBX_0101,Conversation,[啊] 我 跟 你 讲 一个 事 but 不可以 讲 出去 我们 有 ...,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",...,但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,"i tell you one thing but can't tell us, we hav..."
4,4,4,4,4,4,18,01NC01FBX_0101,Conversation,因为 top four [mah],"['因为', 'top', 'four']",...,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名,because top four


In [81]:
csw_df.to_csv('seame_with_eng2_zh2_translations.csv')

In [115]:
csw_df = pd.read_csv('seame_with_eng2_zh2_translations.csv')

In [116]:
csw_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1',
       'file', 'type', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'start_of_code_switches',
       'length_of_code_switched_segments', 'direction_of_code_switches',
       'number_of_code_switches', 'intersentential_code_switch?',
       'type_of_code_switching', 'language', 'conversation', 'speaker',
       'marking', 'first_cs_word', 'first_cs_word_eng_pos', 'word_length_eng',
       'english_translation', 'first_cs_word_zh', 'word_length_zh',
       'first_cs_word_zh_pos', 'first_cs_word_eng_raw_freq',
       'first_cs_word_zh_raw_freq', 'first_cs_word_eng_freq',
       'first_cs_word_zh_freq', 'chinese_translation',
       'chinese_wbw_translation', 'english_wbw_translation'],
      dtype='object')

In [118]:
csw_df = csw_df[['file', 'type', 'conversation', 'speaker', 'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word',
       'first_cs_word_eng_pos', 'word_length_eng', 'english_translation', 'english_wbw_translation', 'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation']]
csw_df.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [119]:
without_todo = csw_df.replace('TODO', np.NaN)
without_todo = without_todo.dropna()
len(without_todo)

28333

In [120]:
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.81279875271622,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549859863469642,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562320373794,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.81279875271622,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091471764630844,34.644711,因为前四名,因为 前 四名


In [89]:
zh_df = df[df['language']=='ZH']
zh_df = zh_df[zh_df['length_of_turn']!=0]
zh_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
2,01NC02FBY_0101,Conversation,我们,['我们'],1,-1,-1,-1,0,-1,-1,ZH,1,2
3,01NC02FBY_0101,Conversation,进 [啊] 没有,"['进', '没有']",2,-1,-1,-1,0,-1,-1,ZH,1,2
4,01NC02FBY_0101,Conversation,(ppl) 等一下 (ppb),"['等', '一下']",2,-1,-1,-1,0,-1,-1,ZH,1,2
9,01NC02FBY_0101,Conversation,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,-1,-1,-1,0,-1,-1,ZH,1,2
10,01NC02FBY_0101,Conversation,算,['算'],1,-1,-1,-1,0,-1,-1,ZH,1,2


In [121]:
zh_df.columns

Index(['file', 'type', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'start_of_code_switches',
       'length_of_code_switched_segments', 'direction_of_code_switches',
       'number_of_code_switches', 'intersentential_code_switch?',
       'type_of_code_switching', 'language', 'conversation', 'speaker'],
      dtype='object')

In [123]:
zh_df = zh_df[['file', 'type', 'conversation', 'speaker', 'transcript_2015', 'filtered_transcript_2015',
       'length_of_turn', 'language']]
zh_df.head()

,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language
2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH
3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH
4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH
9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH
10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH


In [136]:
mean_lengths = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]

    length = 0
    for word in transcript:
        length += len(word)
    mean_lengths.append(length/len(transcript))

zh_df['mean word length'] = mean_lengths

In [147]:
random_length = []
random_word = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]

    word = random.choice(transcript)
    random_word.append(word)
    random_length.append(len(word))

zh_df['random word'] = random_word
zh_df['random word length'] = random_length

In [150]:
tags = []
for word in list(zh_df['random word']):
    doc = nlp(str(word))
    for token in doc:
        tags.append(token.tag_)
        break
        
coded_POS = []
for pos in tags:
    if pos == 'NN':
        coded_POS.append(0)
    elif pos == 'NR':
        coded_POS.append(0)
    elif pos == 'NT':
        coded_POS.append(0)
    elif pos == 'VE':
        coded_POS.append(1)
    elif pos == 'VV':
        coded_POS.append(1)
    else:
        coded_POS.append(2)
        
zh_df['random POS'] = coded_POS

In [160]:
mode_tags = []

for index, row in zh_df.iterrows():
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    
    tags = []
    for word in transcript:
        doc = nlp(str(word))
        for token in doc:
            tags.append(token.tag_)
            break
    coded_POS = []
    for pos in tags:
        if pos == 'NN':
            coded_POS.append(0)
        elif pos == 'NR':
            coded_POS.append(0)
        elif pos == 'NT':
            coded_POS.append(0)
        elif pos == 'VE':
            coded_POS.append(1)
        elif pos == 'VV':
            coded_POS.append(1)
        else:
            coded_POS.append(2)
    
    mode_tags.append(max(set(coded_POS), key=coded_POS.count))

In [161]:
zh_df['mode POS'] = mode_tags

In [171]:
total_count_zh = 26859461025
random_freqs = []

for i in range(len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800


ConnectionError: HTTPSConnectionPool(host='api.phrasefinder.io', port=443): Max retries exceeded with url: /search?corpus=chi&query=%E4%BC%9A&topk=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbfe83f3490>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [172]:
len(random_freqs)

14862

In [175]:
for i in range(14862, len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900


ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [176]:
len(random_freqs)

16922

In [177]:
for i in range(16922, len(list(zh_df['random word']))):
    if i%100 == 0:
        print(i)
    
    word = list(zh_df['random word'])[i]
    encoded_query = urllib.parse.quote(word)
    params = {'corpus': 'chi', 'query': encoded_query, 'topk': 1}
    params = '&'.join('{}={}'.format(name, value) for name, value in params.items())

    response = requests.get('https://api.phrasefinder.io/search?' + params)
    
    if response.status_code == 200:
        if response.json()['phrases'] == []:
            random_freqs.append(-math.log2(1/total_count_zh))
                    
        else:
            raw_freq = response.json()['phrases'][0]['mc']
            random_freqs.append(-math.log2(int(raw_freq)/total_count_zh))
    else:
        random_freqs.append('TODO')
    

17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18600
18700
18800
18900
19000
19100
19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700


In [178]:
print(random_freqs.count('TODO'))

3


In [179]:
zh_df['random word frequency'] = random_freqs

In [180]:
zh_df.to_csv('seame_non_cs_data.csv')

In [2]:
zh_df = pd.read_csv('seame_non_cs_data.csv')
zh_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal
0,0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524,24.420960,24.420960
1,1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251,24.086453,24.284205
2,2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378,23.196542,30.291875
3,3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544,21.718952,24.101382
4,4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016,24.420960,24.420960


In [21]:
# phrasefinder.io discontinued - now have to do manual calculations
# try searching googlebooks 1-grams first, then try 2-grams for frequencies not found in 1-grams

In [5]:
max = 0
for transcript in list(zh_df['filtered_transcript_2015']):
    transcript = transcript.strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        if len(word)>max:
            max = len(word)
            print(word)
print(max)

我们
八十六
三天三夜
#beh-tahan#
#goreng-pisang#
#xiao-ding-dong#
16


In [6]:
num = False
for transcript in list(zh_df['filtered_transcript_2015']):
    transcript = transcript.strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        num = any(char.isdigit() for char in word)

print(num)

False


In [3]:
google_books_df_0 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-0.gz', sep='\t', header=None)
google_books_df_0.head()

,0,1,2,3
0,"0,05_NOUN",1955,1,1
1,"0,05_NOUN",1957,1,1
2,"0,05_NOUN",1963,2,2
3,"0,05_NOUN",1966,1,1
4,"0,05_NOUN",1968,1,1


In [4]:
google_books_df_1 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-1.gz', sep='\t', header=None)
google_books_df_1.head()

,0,1,2,3
0,"1,116",1911,2,1
1,"1,116",1948,1,1
2,"1,116",1949,5,2
3,"1,116",1954,1,1
4,"1,116",1956,2,2


In [5]:
google_books_df = pd.concat([google_books_df_0, google_books_df_1], axis=0)

In [6]:
len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)

True

In [7]:
len(google_books_df)

4268208

In [8]:
google_books_df_2 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-2.gz', sep='\t', header=None)

In [9]:
google_books_df = pd.concat([google_books_df, google_books_df_2], axis=0)
#len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)+len(google_books_df_2)

In [10]:
google_books_df_3 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-3.gz', sep='\t', header=None)
google_books_df_4 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-4.gz', sep='\t', header=None)
google_books_df_5 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-5.gz', sep='\t', header=None)
google_books_df_6 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-6.gz', sep='\t', header=None)
google_books_df_7 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-7.gz', sep='\t', header=None)
google_books_df_8 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-8.gz', sep='\t', header=None)
google_books_df_9 = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-9.gz', sep='\t', header=None)

In [11]:
google_books_df = pd.concat([google_books_df, google_books_df_3, google_books_df_4, google_books_df_5, google_books_df_6, google_books_df_7, google_books_df_8, google_books_df_9], axis=0)

In [12]:
len(google_books_df) == len(google_books_df_0)+len(google_books_df_1)+len(google_books_df_2)+len(google_books_df_3)+len(google_books_df_4)+len(google_books_df_5)+len(google_books_df_6)+len(google_books_df_7)+len(google_books_df_8)+len(google_books_df_9)

True

In [13]:
google_books_df_a = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-a.gz', sep='\t', header=None)
google_books_df_b = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-b.gz', sep='\t', header=None)
google_books_df_c = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-c.gz', sep='\t', header=None)
google_books_df_d = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-d.gz', sep='\t', header=None)
google_books_df_e = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-e.gz', sep='\t', header=None)
google_books_df_f = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-f.gz', sep='\t', header=None)
google_books_df_g = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-g.gz', sep='\t', header=None)
google_books_df_h = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-h.gz', sep='\t', header=None)
google_books_df_i = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-i.gz', sep='\t', header=None)
google_books_df_j = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-j.gz', sep='\t', header=None)
google_books_df_k = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-k.gz', sep='\t', header=None)
google_books_df_l = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-l.gz', sep='\t', header=None)
google_books_df_m = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-m.gz', sep='\t', header=None)
google_books_df_n = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-n.gz', sep='\t', header=None)
google_books_df_o = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-o.gz', sep='\t', header=None)
google_books_df_p = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-p.gz', sep='\t', header=None)
google_books_df_q = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-q.gz', sep='\t', header=None)
google_books_df_r = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-r.gz', sep='\t', header=None)
google_books_df_s = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-s.gz', sep='\t', header=None)
google_books_df_t = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-t.gz', sep='\t', header=None)
google_books_df_u = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-u.gz', sep='\t', header=None)
google_books_df_v = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-v.gz', sep='\t', header=None)
google_books_df_w = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-w.gz', sep='\t', header=None)
google_books_df_x = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-x.gz', sep='\t', header=None)
google_books_df_y = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-y.gz', sep='\t', header=None)
google_books_df_z = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-z.gz', sep='\t', header=None)
google_books_df_other = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-other.gz', sep='\t', header=None)
#google_books_df_pos = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-pos.gz', sep='\t', header=None)
#google_books_df_punctuation = pd.read_csv('googlebooks-chi-sim-all-1gram-20120701-punctuation.gz', sep='\t', header=None)

In [14]:
google_books_df = pd.concat([google_books_df, google_books_df_a, google_books_df_b, google_books_df_c, google_books_df_d, google_books_df_e, google_books_df_f, google_books_df_g, google_books_df_h, google_books_df_i, google_books_df_j, google_books_df_k, google_books_df_l, google_books_df_m, google_books_df_n, google_books_df_o, google_books_df_p, google_books_df_q, google_books_df_r, google_books_df_s, google_books_df_t, google_books_df_u, google_books_df_v, google_books_df_w, google_books_df_x, google_books_df_y, google_books_df_z, google_books_df_other], axis=0)

In [15]:
google_books_df = google_books_df.rename(columns={0: "ngram", 1: "year", 2: "match_count", 3: "volume_count"})

In [16]:
google_books_df.head()

,ngram,year,match_count,volume_count
0,"0,05_NOUN",1955,1,1
1,"0,05_NOUN",1957,1,1
2,"0,05_NOUN",1963,2,2
3,"0,05_NOUN",1966,1,1
4,"0,05_NOUN",1968,1,1


In [17]:
len(google_books_df)

24698298

In [18]:
cleaned_ngram = [str(word)[:(str(word).index('_'))] if '_' in str(word) else str(word) for word in list(google_books_df['ngram'])]
google_books_df['cleaned_ngram'] = cleaned_ngram

In [19]:
google_books_df.head()

,ngram,year,match_count,volume_count,cleaned_ngram
0,"0,05_NOUN",1955,1,1,"0,05"
1,"0,05_NOUN",1957,1,1,"0,05"
2,"0,05_NOUN",1963,2,2,"0,05"
3,"0,05_NOUN",1966,1,1,"0,05"
4,"0,05_NOUN",1968,1,1,"0,05"


In [20]:
google_books_df.tail()

,ngram,year,match_count,volume_count,cleaned_ngram
1096,Ⅲ,2003,5,4,Ⅲ
1097,Ⅲ,2005,17,16,Ⅲ
1098,Ⅲ,2006,10,6,Ⅲ
1099,Ⅲ,2007,12,4,Ⅲ
1100,Ⅲ,2008,1,1,Ⅲ


In [21]:
match_count_int = [int(num) for num in list(google_books_df['match_count'])]
google_books_df['match_count_as_int'] = match_count_int
google_books_df.head()

,ngram,year,match_count,volume_count,cleaned_ngram,match_count_as_int
0,"0,05_NOUN",1955,1,1,"0,05",1
1,"0,05_NOUN",1957,1,1,"0,05",1
2,"0,05_NOUN",1963,2,2,"0,05",2
3,"0,05_NOUN",1966,1,1,"0,05",1
4,"0,05_NOUN",1968,1,1,"0,05",1


In [26]:
total_count_zh = 26859461025
unique_seame_words = defaultdict()
for index, row in zh_df.iterrows():
    #print(str(index/22774*100)+'% done')
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    for word in transcript:
        if len(word)>2: #doing this to stick to using 1-gram data (bigram data is too large)
            word = word[:2]
        unique_seame_words[word] = 1/total_count_zh

In [27]:
len(list(unique_seame_words.keys()))

7167

In [29]:
for i in range(len(list(unique_seame_words.keys()))):
    print(str(i/len(list(unique_seame_words.keys()))*100)+'% done')
    word = list(unique_seame_words.keys())[i]
    sub_df = google_books_df[google_books_df['cleaned_ngram']==word]
    word_freq_total = sub_df['match_count_as_int'].sum()
    if word_freq_total != 0:
        unique_seame_words[word] = word_freq_total/total_count_zh
    

0.0% done
0.013952839402818475% done
0.02790567880563695% done
0.041858518208455424% done
0.0558113576112739% done
0.06976419701409237% done
0.08371703641691085% done
0.09766987581972932% done
0.1116227152225478% done
0.12557555462536626% done
0.13952839402818473% done
0.1534812334310032% done
0.1674340728338217% done
0.18138691223664016% done
0.19533975163945863% done
0.2092925910422771% done
0.2232454304450956% done
0.23719826984791403% done
0.25115110925073253% done
0.26510394865355097% done
0.27905678805636946% done
0.2930096274591879% done
0.3069624668620064% done
0.3209153062648249% done
0.3348681456676434% done
0.34882098507046183% done
0.3627738244732803% done
0.37672666387609877% done
0.39067950327891726% done
0.40463234268173576% done
0.4185851820845542% done
0.43253802148737264% done
0.4464908608901912% done
0.46044370029300963% done
0.47439653969582807% done
0.48834937909864656% done
0.5023022185014651% done
0.5162550579042835% done
0.5302078973071019% done
0.54416073670992

4.66024836054137% done
4.674201199944188% done
4.6881540393470065% done
4.7021068787498255% done
4.716059718152644% done
4.730012557555463% done
4.743965396958281% done
4.7579182363611% done
4.771871075763918% done
4.785823915166736% done
4.799776754569555% done
4.813729593972373% done
4.827682433375192% done
4.84163527277801% done
4.855588112180829% done
4.869540951583647% done
4.883493790986466% done
4.897446630389284% done
4.911399469792102% done
4.925352309194921% done
4.9393051485977395% done
4.9532579880005585% done
4.967210827403377% done
4.981163666806196% done
4.995116506209014% done
5.009069345611832% done
5.023022185014651% done
5.036975024417469% done
5.050927863820288% done
5.064880703223106% done
5.078833542625925% done
5.092786382028742% done
5.106739221431561% done
5.120692060834379% done
5.134644900237198% done
5.148597739640016% done
5.1625505790428345% done
5.1765034184456535% done
5.190456257848472% done
5.204409097251291% done
5.218361936654109% done
5.232314776056

9.446072275708106% done
9.460025115110925% done
9.473977954513742% done
9.487930793916561% done
9.50188363331938% done
9.5158364727222% done
9.529789312125017% done
9.543742151527836% done
9.557694990930655% done
9.571647830333472% done
9.58560066973629% done
9.59955350913911% done
9.613506348541929% done
9.627459187944746% done
9.641412027347565% done
9.655364866750384% done
9.669317706153201% done
9.68327054555602% done
9.69722338495884% done
9.711176224361658% done
9.725129063764475% done
9.739081903167294% done
9.753034742570113% done
9.766987581972932% done
9.78094042137575% done
9.794893260778569% done
9.808846100181388% done
9.822798939584205% done
9.836751778987024% done
9.850704618389843% done
9.864657457792662% done
9.878610297195479% done
9.892563136598298% done
9.906515976001117% done
9.920468815403934% done
9.934421654806753% done
9.948374494209572% done
9.962327333612391% done
9.976280173015208% done
9.990233012418027% done
10.004185851820846% done
10.018138691223664% don

14.07841495744384% done
14.092367796846657% done
14.106320636249478% done
14.120273475652295% done
14.134226315055114% done
14.148179154457932% done
14.162131993860749% done
14.17608483326357% done
14.190037672666387% done
14.203990512069206% done
14.217943351472023% done
14.231896190874844% done
14.245849030277661% done
14.25980186968048% done
14.273754709083297% done
14.287707548486118% done
14.301660387888933% done
14.315613227291754% done
14.329566066694571% done
14.34351890609739% done
14.357471745500208% done
14.371424584903028% done
14.385377424305846% done
14.399330263708665% done
14.413283103111482% done
14.427235942514303% done
14.44118878191712% done
14.455141621319939% done
14.469094460722756% done
14.483047300125577% done
14.497000139528392% done
14.510952978931213% done
14.52490581833403% done
14.53885865773685% done
14.552811497139666% done
14.566764336542487% done
14.580717175945304% done
14.594670015348123% done
14.60862285475094% done
14.622575694153761% done
14.63652

18.710757639179572% done
18.72471047858239% done
18.73866331798521% done
18.752616157388026% done
18.76656899679085% done
18.780521836193664% done
18.794474675596483% done
18.808427514999302% done
18.82238035440212% done
18.83633319380494% done
18.85028603320776% done
18.864238872610574% done
18.878191712013397% done
18.892144551416212% done
18.90609739081903% done
18.92005023022185% done
18.93400306962467% done
18.947955909027485% done
18.961908748430307% done
18.975861587833123% done
18.98981442723594% done
19.00376726663876% done
19.01772010604158% done
19.0316729454444% done
19.045625784847218% done
19.059578624250033% done
19.073531463652856% done
19.08748430305567% done
19.10143714245849% done
19.11538998186131% done
19.129342821264128% done
19.143295660666944% done
19.157248500069766% done
19.17120133947258% done
19.1851541788754% done
19.19910701827822% done
19.21305985768104% done
19.227012697083858% done
19.240965536486677% done
19.254918375889492% done
19.268871215292315% do

...

80.29859076322032% done
80.31254360262314% done
80.32649644202596% done
80.34044928142877% done
80.3544021208316% done
80.36835496023441% done
80.38230779963723% done
80.39626063904004% done
80.41021347844286% done
80.42416631784567% done
80.4381191572485% done
80.45207199665131% done
80.46602483605415% done
80.47997767545696% done
80.49393051485978% done
80.50788335426259% done
80.52183619366541% done
80.53578903306823% done
80.54974187247105% done
80.56369471187386% done
80.57764755127668% done
80.5916003906795% done
80.60555323008232% done
80.61950606948514% done
80.63345890888796% done
80.64741174829078% done
80.6613645876936% done
80.67531742709642% done
80.68927026649924% done
80.70322310590205% done
80.71717594530486% done
80.73112878470768% done
80.7450816241105% done
80.75903446351333% done
80.77298730291615% done
80.78694014231897% done
80.80089298172179% done
80.8148458211246% done
80.82879866052741% done
80.84275149993023% done
80.85670433933305% done
80.87065717873587% don

85.09836751778988% done
85.11232035719269% done
85.1262731965955% done
85.14022603599832% done
85.15417887540114% done
85.16813171480396% done
85.18208455420678% done
85.1960373936096% done
85.20999023301242% done
85.22394307241524% done
85.23789591181806% done
85.25184875122088% done
85.2658015906237% done
85.27975443002651% done
85.29370726942933% done
85.30766010883215% done
85.32161294823497% done
85.33556578763778% done
85.3495186270406% done
85.36347146644341% done
85.37742430584623% done
85.39137714524905% done
85.40532998465189% done
85.4192828240547% done
85.43323566345752% done
85.44718850286033% done
85.46114134226315% done
85.47509418166597% done
85.48904702106879% done
85.5029998604716% done
85.51695269987442% done
85.53090553927724% done
85.54485837868006% done
85.55881121808288% done
85.5727640574857% done
85.58671689688852% done
85.60066973629134% done
85.61462257569416% done
85.62857541509698% done
85.6425282544998% done
85.65648109390261% done
85.67043393330542% done


89.91209711176225% done
89.92604995116507% done
89.94000279056789% done
89.9539556299707% done
89.96790846937351% done
89.98186130877633% done
89.99581414817915% done
90.00976698758197% done
90.02371982698479% done
90.0376726663876% done
90.05162550579044% done
90.06557834519325% done
90.07953118459606% done
90.09348402399888% done
90.1074368634017% done
90.12138970280452% done
90.13534254220734% done
90.14929538161016% done
90.16324822101298% done
90.1772010604158% done
90.19115389981862% done
90.20510673922143% done
90.21905957862425% done
90.23301241802707% done
90.24696525742989% done
90.26091809683271% done
90.27487093623552% done
90.28882377563833% done
90.30277661504115% done
90.31672945444397% done
90.33068229384679% done
90.34463513324962% done
90.35858797265244% done
90.37254081205526% done
90.38649365145807% done
90.40044649086089% done
90.4143993302637% done
90.42835216966652% done
90.44230500906934% done
90.45625784847216% done
90.47021068787498% done
90.4841635272778% don

94.72582670573462% done
94.73977954513744% done
94.75373238454026% done
94.76768522394308% done
94.78163806334588% done
94.7955909027487% done
94.80954374215152% done
94.82349658155435% done
94.83744942095717% done
94.85140226035999% done
94.86535509976281% done
94.87930793916563% done
94.89326077856843% done
94.90721361797125% done
94.92116645737407% done
94.93511929677689% done
94.94907213617971% done
94.96302497558253% done
94.97697781498535% done
94.99093065438817% done
95.00488349379098% done
95.0188363331938% done
95.03278917259662% done
95.04674201199944% done
95.06069485140226% done
95.07464769080508% done
95.0886005302079% done
95.10255336961072% done
95.11650620901352% done
95.13045904841636% done
95.14441188781917% done
95.158364727222% done
95.17231756662481% done
95.18627040602763% done
95.20022324543045% done
95.21417608483327% done
95.22812892423607% done
95.2420817636389% done
95.25603460304171% done
95.26998744244453% done
95.28394028184735% done
95.29789312125018% don

99.52560346030417% done
99.53955629970699% done
99.5535091391098% done
99.56746197851263% done
99.58141481791544% done
99.59536765731826% done
99.60932049672108% done
99.6232733361239% done
99.63722617552672% done
99.65117901492954% done
99.66513185433236% done
99.67908469373518% done
99.693037533138% done
99.70699037254082% done
99.72094321194363% done
99.73489605134644% done
99.74884889074926% done
99.76280173015209% done
99.77675456955491% done
99.79070740895773% done
99.80466024836055% done
99.81861308776337% done
99.83256592716619% done
99.84651876656899% done
99.86047160597181% done
99.87442444537463% done
99.88837728477745% done
99.90233012418027% done
99.91628296358309% done
99.9302358029859% done
99.94418864238874% done
99.95814148179154% done
99.97209432119436% done
99.98604716059718% done


In [30]:
unique_seame_words

defaultdict(None,
            {'我们': 0.005448909040422564,
             '进': 0.0008321377699722476,
             '没有': 0.0032778549769875735,
             '等': 0.0029429024628017456,
             '一下': 0.00021130766528476905,
             '拿': 0.00011222228164572784,
             '多少': 0.00022451314247844256,
             '分': 0.0013238817401027875,
             '到': 0.006308355623453915,
             '最后': 0.000497524205253482,
             '算': 0.00014937030926516888,
             '一百': 5.748439994990555e-08,
             '八十': 4.644918223931487e-07,
             '加': 0.0006283989832964268,
             '一': 0.0156901793974103,
             '六': 0.0012902805446372504,
             '八': 0.0011413986293866817,
             '我': 0.006412479864718357,
             '五': 0.002033884594674215,
             '就': 0.004338209053843067,
             '了': 0.020931112187125504,
             '三': 0.0036911536649123808,
             '九': 0.00178264217422062,
             '一八': 3.723082898309945e-11

In [31]:
mean_freqs = []

for index, row in zh_df.iterrows():
    print(str(index/22774*100)+'% done')
    
    transcript = row['filtered_transcript_2015'].strip('][').split(', ')
    transcript = [word[1:-1] for word in transcript if word.startswith("'") and word.endswith("'")]
    
    freqs = []
    for word in transcript:
        if len(word)>2: #doing this to stick to using 1-gram data (bigram data is too large)
            word = word[:2]
        freqs.append(unique_seame_words[word])
        
    try:
        mean_freqs.append(-math.log2(np.mean(freqs)))
    except:
        mean_freqs.append('TODO')

print(mean_freqs.count('TODO'))
        
zh_df['mean word frequency'] = mean_freqs

0.0% done
0.004390972161236498% done
0.008781944322472996% done
0.013172916483709492% done
0.017563888644945992% done
0.02195486080618249% done
0.026345832967418985% done
0.030736805128655485% done
0.035127777289891984% done
0.03951874945112848% done
0.04390972161236498% done
0.04830069377360148% done
0.05269166593483797% done
0.05708263809607447% done
0.06147361025731097% done
0.06586458241854747% done
0.07025555457978397% done
0.07464652674102046% done
0.07903749890225696% done
0.08342847106349346% done
0.08781944322472995% done
0.09221041538596646% done
0.09660138754720296% done
0.10099235970843945% done
0.10538333186967594% done
0.10977430403091246% done
0.11416527619214895% done
0.11855624835338545% done
0.12294722051462194% done
0.12733819267585844% done
0.13172916483709493% done
0.13612013699833142% done
0.14051110915956794% done
0.14490208132080443% done
0.14929305348204092% done
0.1536840256432774% done
0.15807499780451392% done
0.1624659699657504% done
0.16685694212698693% do

6.516202687274962% done
6.5205936594362% done
6.524984631597436% done
6.529375603758672% done
6.533766575919908% done
6.538157548081146% done
6.542548520242382% done
6.546939492403618% done
6.551330464564854% done
6.55572143672609% done
6.560112408887328% done
6.5645033810485645% done
6.5688943532098% done
6.573285325371037% done
6.577676297532274% done
6.5820672696935105% done
6.586458241854746% done
6.590849214015983% done
6.59524018617722% done
6.599631158338457% done
6.6040221304996924% done
6.608413102660929% done
6.612804074822165% done
6.617195046983403% done
6.6215860191446385% done
6.625976991305875% done
6.630367963467111% done
6.634758935628349% done
6.639149907789585% done
6.643540879950821% done
6.647931852112057% done
6.652322824273293% done
6.656713796434531% done
6.661104768595767% done
6.665495740757003% done
6.66988671291824% done
6.674277685079478% done
6.6786686572407135% done
6.683059629401949% done
6.687450601563186% done
6.691841573724422% done
6.6962325458856595

13.098269956968473% done
13.102660929129708% done
13.107051901290944% done
13.11144287345218% done
13.11583384561342% done
13.120224817774655% done
13.124615789935893% done
13.129006762097129% done
13.133397734258365% done
13.1377887064196% done
13.142179678580836% done
13.146570650742074% done
13.150961622903312% done
13.155352595064548% done
13.159743567225785% done
13.164134539387021% done
13.168525511548257% done
13.172916483709493% done
13.177307455870729% done
13.181698428031966% done
13.186089400193202% done
13.19048037235444% done
13.194871344515677% done
13.199262316676913% done
13.203653288838149% done
13.208044260999385% done
13.21243523316062% done
13.216826205321858% done
13.221217177483094% done
13.22560814964433% done
13.22999912180557% done
13.234390093966805% done
13.238781066128041% done
13.243172038289277% done
13.247563010450513% done
13.25195398261175% done
13.256344954772986% done
13.260735926934222% done
13.265126899095458% done
13.269517871256697% done
13.273908

19.68472819882322% done
19.689119170984455% done
19.693510143145694% done
19.69790111530693% done
19.702292087468166% done
19.706683059629402% done
19.711074031790638% done
19.715465003951874% done
19.719855976113113% done
19.72424694827435% done
19.728637920435585% done
19.73302889259682% done
19.737419864758056% done
19.741810836919292% done
19.746201809080528% done
19.750592781241767% done
19.754983753403003% done
19.759374725564243% done
19.76376569772548% done
19.768156669886714% done
19.77254764204795% done
19.776938614209186% done
19.781329586370422% done
19.785720558531658% done
19.790111530692894% done
19.794502502854133% done
19.79889347501537% done
19.803284447176605% done
19.80767541933784% done
19.812066391499076% done
19.816457363660316% done
19.82084833582155% done
19.825239307982788% done
19.829630280144023% done
19.834021252305263% done
19.8384122244665% done
19.842803196627735% done
19.84719416878897% done
19.851585140950206% done
19.855976113111442% done
19.860367085

25.234917010626152% done
25.23930798278739% done
25.243698954948623% done
25.248089927109863% done
25.252480899271095% done
25.256871871432335% done
25.261262843593574% done
25.265653815754806% done
25.270044787916046% done
25.274435760077278% done
25.278826732238517% done
25.283217704399753% done
25.287608676560993% done
25.29199964872223% done
25.296390620883464% done
25.300781593044704% done
25.305172565205936% done
25.309563537367175% done
25.313954509528408% done
25.318345481689647% done
25.32273645385088% done
25.32712742601212% done
25.33151839817336% done
25.33590937033459% done
25.34030034249583% done
25.344691314657062% done
25.3490822868183% done
25.353473258979538% done
25.357864231140777% done
25.36225520330201% done
25.36664617546325% done
25.371037147624488% done
25.37542811978572% done
25.37981909194696% done
25.384210064108192% done
25.38860103626943% done
25.392992008430664% done
25.397382980591903% done
25.40177395275314% done
25.406164924914375% done
25.410555897075

...

74.52357952050585% done
74.52797049266707% done
74.53236146482831% done
74.53675243698954% done
74.54114340915079% done
74.54553438131202% done
74.54992535347326% done
74.55431632563449% done
74.55870729779573% done
74.56309826995697% done
74.5674892421182% done
74.57188021427945% done
74.57627118644068% done
74.58066215860192% done
74.58505313076314% done
74.58944410292439% done
74.59383507508562% done
74.59822604724687% done
74.6026170194081% done
74.60700799156933% done
74.61139896373057% done
74.61578993589181% done
74.62018090805304% done
74.62457188021428% done
74.62896285237551% done
74.63335382453675% done
74.63774479669799% done
74.64213576885923% done
74.64652674102045% done
74.6509177131817% done
74.65530868534293% done
74.65969965750418% done
74.66409062966541% done
74.66848160182664% done
74.67287257398787% done
74.67726354614912% done
74.68165451831035% done
74.68604549047159% done
74.69043646263283% done
74.69482743479406% done
74.69921840695531% done
74.70360937911653% 

80.39430930007903% done
80.39870027224028% done
80.40309124440151% done
80.40748221656274% done
80.41187318872399% done
80.41626416088522% done
80.42065513304647% done
80.4250461052077% done
80.42943707736893% done
80.43382804953016% done
80.43821902169141% done
80.44260999385264% done
80.44700096601387% done
80.4513919381751% done
80.45578291033635% done
80.46017388249759% done
80.46456485465883% done
80.46895582682005% done
80.4733467989813% done
80.47773777114253% done
80.48212874330378% done
80.486519715465% done
80.49091068762624% done
80.49530165978747% done
80.49969263194872% done
80.50408360410995% done
80.50847457627118% done
80.51286554843242% done
80.51725652059366% done
80.5216474927549% done
80.52603846491613% done
80.53042943707737% done
80.5348204092386% done
80.53921138139984% done
80.54360235356108% done
80.54799332572232% done
80.55238429788355% done
80.5567752700448% done
80.56116624220603% done
80.56555721436726% done
80.56994818652849% done
80.57433915868974% done


86.64266268551857% done
86.6470536576798% done
86.65144462984105% done
86.65583560200228% done
86.66022657416352% done
86.66461754632475% done
86.669008518486% done
86.67339949064723% done
86.67779046280847% done
86.6821814349697% done
86.68657240713094% done
86.69096337929217% done
86.69535435145342% done
86.69974532361465% done
86.70413629577588% done
86.70852726793711% done
86.71291824009836% done
86.7173092122596% done
86.72170018442084% done
86.72609115658206% done
86.7304821287433% done
86.73487310090454% done
86.73926407306578% done
86.743655045227% done
86.74804601738825% done
86.75243698954948% done
86.75682796171073% done
86.76121893387196% done
86.76560990603319% done
86.77000087819444% done
86.77439185035567% done
86.7787828225169% done
86.78317379467813% done
86.78756476683938% done
86.79195573900061% done
86.79634671116186% done
86.80073768332309% done
86.80512865548432% done
86.80951962764556% done
86.8139105998068% done
86.81830157196804% done
86.82269254412927% done
86

92.6495126020901% done
92.65390357425134% done
92.65829454641258% done
92.66268551857382% done
92.66707649073504% done
92.67146746289629% done
92.67585843505752% done
92.68024940721877% done
92.68464037937999% done
92.68903135154123% done
92.69342232370246% done
92.69781329586371% done
92.70220426802494% done
92.70659524018618% done
92.71098621234741% done
92.71537718450865% done
92.71976815666989% done
92.72415912883112% done
92.72855010099235% done
92.7329410731536% done
92.73733204531483% done
92.74172301747608% done
92.74611398963731% done
92.75050496179854% done
92.75489593395977% done
92.75928690612102% done
92.76367787828225% done
92.76806885044348% done
92.77245982260473% done
92.77685079476596% done
92.78124176692721% done
92.78563273908843% done
92.79002371124967% done
92.7944146834109% done
92.79880565557215% done
92.80319662773337% done
92.80758759989462% done
92.81197857205585% done
92.8163695442171% done
92.82076051637833% done
92.82515148853956% done
92.8295424607008% do

98.8451743215948% done
98.84956529375603% done
98.85395626591728% done
98.85834723807851% done
98.86273821023974% done
98.86712918240099% done
98.87152015456222% done
98.87591112672347% done
98.8803020988847% done
98.88469307104593% done
98.88908404320716% done
98.89347501536841% done
98.89786598752964% done
98.90225695969087% done
98.9066479318521% done
98.91103890401335% done
98.91542987617458% done
98.91982084833583% done
98.92421182049705% done
98.9286027926583% done
98.93299376481953% done
98.93738473698077% done
98.941775709142% done
98.94616668130324% done
98.95055765346447% done
98.95494862562572% done
98.95933959778695% done
98.96373056994818% done
98.96812154210942% done
98.97251251427066% done
98.9769034864319% done
98.98129445859313% done
98.98568543075437% done
98.9900764029156% done
98.99446737507685% done
98.99885834723808% done
99.00324931939932% done
99.00764029156055% done
99.0120312637218% done
99.01642223588303% done
99.02081320804426% done
99.02520418020549% done
9

In [32]:
len(mean_freqs)

22773

In [33]:
zh_df.to_csv('seame_non_cs_data_mean_freqs.csv') #rename this file if all looks good
zh_df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency
0,0,2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2,8.519816876703524,24.420960,24.420960,7.519817
1,1,3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1,11.230889976662251,24.086453,24.284205,8.926648
2,2,4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1,13.208367276887378,23.196542,30.291875,9.308506
3,3,9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1,11.972945663039544,21.718952,24.101382,8.752136
4,4,10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1,13.708818971267016,24.420960,24.420960,12.708819


In [34]:
zh_df.to_csv('seame_non_cs_data.csv')

In [6]:
transcripts_as_lst = list(zh_df['filtered_transcript_2015'])
cleaned_transcripts = []
for transcript in transcripts_as_lst:
    cleaned_transcript = transcript.strip('][').split(', ')
    cleaned_transcript = [word[1:-1] for word in cleaned_transcript if word.startswith("'") and word.endswith("'")]
    cleaned_transcripts.append(cleaned_transcript)
cleaned_transcripts
transcripts_as_str = [' '.join(lst) for lst in cleaned_transcripts]
transcripts_as_str

['我们',
 '进 没有',
 '等 一下',
 '我们 拿 多少 分 到 最后',
 '算',
 '一百',
 '八十六 加 八十 加',
 '一 六',
 '八十六 加',
 '没有 没有 八 八十 没有 算',
 '我 拿 一 五 八',
 '就 八十 我 拿 了 一 三 九',
 '一 三 九 八十六',
 '一八 四',
 '没有 我 拿 比较 高 一点点 一 五 八',
 '一 七八',
 '你 拿 多少 一 五一',
 '还是 我 是',
 '十几 号 讲 多 一次 十一月 十号',
 '十月 十一号',
 '五 六七',
 '我们',
 '十 号 十月 对 不 对',
 '十号 十月',
 '十月 十号 听 起来 像是 很 没有 空 的 时候',
 '对 拜 六',
 '很 早 很 早 我 忘记 了 去年 几时',
 '就 给 我们 射 先',
 '这样',
 '会 有 失手 的 时候',
 '赢 我们 二十多',
 '什么 意思',
 '他们 才 五六 多',
 '对 他们 的 五六 多 是',
 '这样 而已 我 还 以为 是 那种 过 六头 的',
 '去年 你们 你 的 那个 一七零 加 他们 的 一 就 过 了 六百 了',
 '对 对 对 就 连 做 三天三夜',
 '我们 呃 我 我 讲 若惠',
 '他 就 讲',
 '他 就 讲',
 '差不多 若惠 她 讲 她 六点 多',
 '我们',
 '我 没有 驾车',
 '是 你 跟 我 讲 他 有 驾车',
 '八点',
 '我 就 看到',
 '庆辉 就 问',
 '他 就 会',
 '他 就 碎碎念 碎碎念',
 '她 就 讲',
 '呃',
 '他 就 讲',
 '诶',
 '呃 对 对',
 '对 对 对',
 '讲 了 什么 东西',
 '我 忘记 了 了',
 '对',
 '我 就 #beh-tahan# 他',
 '刚才 瑞宝 跟 我 讲',
 '对',
 '对 对',
 '那个 叫 什么 名',
 '那个 叫 什么 名 我们 那个 对',
 '不 认识',
 '因为 他 每次 呃',
 '呃 读 不 完',
 '就 本来 是',
 '没有 没有 他 有',
 '屁',
 '我 没有 什么 打算 要 做',
 '忘记 了',
 '他们',
 '屁 没有 打包 他 没有 

In [7]:
file = open('non_cs_transcripts.txt','w')
for line in transcripts_as_str:
    file.write(line +"\n")
file.close()

In [ ]:
#what to do about no cs points in non-cs sentences? just choose a random word?

#compare surp distribution of zh cs1 to random non-cs
#compare surp distribution of random non-cs to held out zh wiki 
#logistic regression zh cs1 versus random non-cs
#compare eng cs1 to monolingual wiki english: length, pos, freq, surp (use same ml eng data as before)
#compare eng cs1/ml eng to zh cs1/ml zh (use same ml data as before)

#do all again using other training data for models?

# NOTE: translations in both directions are pretty bad on this data set. 

# TODO: reorder columns + rename to match previous analyses

In [169]:
len(random_freqs)

6525

In [170]:
len(mean_freqs)

5632

In [164]:
zh_df.head()

,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,mean word length,random word,random word length,random POS,mode POS
2,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,2.0,我们,2,2,2
3,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,1.5,进,1,1,1
4,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,1.5,一下,2,2,1
9,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,1.5,最后,2,2,1
10,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,1.0,算,1,1,1


In [165]:
len(zh_df)

22773

In [166]:
zh_df.to_csv('seame_non_cs_data.csv')

In [91]:
en_df = df[df['language']=='EN']
en_df = en_df[en_df['length_of_turn']!=0]
en_df.head()

,file,type,transcript_2015,filtered_transcript_2015,length_of_turn,start_of_code_switches,length_of_code_switched_segments,direction_of_code_switches,number_of_code_switches,intersentential_code_switch?,type_of_code_switching,language,conversation,speaker
6,01NC01FBX_0101,Conversation,[ah] final saturday,"['final', 'saturday']",2,-1,-1,-1,0,-1,-1,EN,1,1
26,01NC01FBX_0101,Conversation,area five,"['area', 'five']",2,-1,-1,-1,0,-1,-1,EN,1,1
38,01NC02FBY_0101,Conversation,ya [lor],['ya'],1,-1,-1,-1,0,-1,-1,EN,1,2
40,01NC02FBY_0101,Conversation,[oh] my god,"['my', 'god']",2,-1,-1,-1,0,-1,-1,EN,1,2
57,01NC02FBY_0101,Conversation,not very so wish,"['not', 'very', 'so', 'wish']",4,-1,-1,-1,0,-1,-1,EN,1,2


In [92]:
len(en_df)

22529